<a href="https://colab.research.google.com/github/chinmaydhama/Personal-projects/blob/main/RealtimeStockNewsSentimentAnalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen, Request

In [ ]:
def get_ticks(url):
    req = Request(url=url,headers={"User-Agent": "Chrome"}) 
    response = urlopen(req)    
    html = BeautifulSoup(response,"html.parser")
    news_table = html.find(class_ = 'page')
    
    ticks = list()
    stocks = list()
    for name_box in news_table.find_all('a', href = True):
        stocks.append(name_box.text.strip())
        ticks.append(name_box['href'].split('-')[-1].strip())
    d = {'stock':stocks, 'tick':ticks}
    df = pd.DataFrame(data = d)
    return df

In [ ]:
url = "https://www.tickertape.in/stocks"

tick_df = get_ticks(url)

In [ ]:
tick_df

,stock,tick
0,ABB India Ltd,ABB
1,Adani Enterprises Ltd,ADEL
2,Adani Green Energy Ltd,ADNA
3,Adani Ports and Special Economic Zone Ltd,APSE
4,Adani Power Ltd,ADAN
...,...,...
95,United Spirits Ltd,UNSP
96,Varun Beverages Ltd,VARB
97,Vedanta Ltd,VDAN
98,Wipro Ltd,WIPR


In [ ]:
def get_data(url):
    req = Request(url=url,headers={"User-Agent": "Chrome"}) 
    response = urlopen(req).read()    
    html = BeautifulSoup(response,"html.parser")
    news_table = html.find(class_ = 'latest-news-holder')
    
    news = list()
    
    for name_box in news_table.find_all('p', class_='shave-root'):
        news.append(name_box.text.strip())
    
    return news

In [ ]:
news = list()
for i, tick in enumerate(tick_df['tick']):
    url = "https://www.tickertape.in/stocks/" + tick + "/news?checklist=basic&ref=stock-overview_overview-sections&type=news"
    headlines = get_data(url)
    news.append(headlines)

In [ ]:
for i, n in enumerate(news):
    print(i+1, ")", n)
    print('-------')

1 ) ['ABB Q2 PAT seen up 48.1% YoY to Rs. 177.7 cr: Prabhudas Lilladher', 'Tube Investment, ABB, Varun Bev, IHCL likely to enter MSCI index']
-------
2 ) ['Artson Engineering spurts after bagging order worth Rs 42.76 cr', 'Street signs: Mkts to feel selling pressure, Adani Enterprises stock & more', 'Movers & Shakers: Stocks that will see action this week']
-------
3 ) ['Adani Green Energy shares rise after firm buys three SPVs from Inox Green Energy', 'Adani Green Energy completes acquisition of three SPVs from Inox Green', 'Inox Wind transfers 3 SPVs to Adani Green Energy; stock zooms ~2.5%']
-------
4 ) ['Taking Stock | Market ends lower ahead of Fed meet outcome; FMCG gains, power stocks drag', 'Stocks in news: TCS, Adani Ports, Bajaj Auto, Infosys and more', 'Adani Ports gets NCLT nod for acquisition of Gangavaram port']
-------
5 ) ['Adani Wilmar, Adani Power freeze at 5% upper circuit; Ambuja Cements up 4%', 'Adani Power,\xa0Adani Wilmar stocks hit 5% upper circuit today']
-----

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 14.4 MB/s 
     |████████████████████████████████| 7.6 MB 58.0 MB/s 
     |████████████████████████████████| 163 kB 64.3 MB/s 


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

Downloading:   0%|          | 0.00/533 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/439M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
labels = {0:'neutral', 1:'positive',2:'negative'}

In [ ]:
def detect(news):
    tot_val = list()
    for n in news:
        if len(n) == 0:
            tot_val.append(['neutral'])
        else:
            inputs = tokenizer(n, return_tensors="pt", padding=True)
            outputs = finbert(**inputs)[0]
            val = list()
            for idx, sent in enumerate(n):
                val.append(labels[np.argmax(outputs.detach().numpy()[idx])])
                print(sent, '----', val[idx])
            print('#######################################################')    
            tot_val.append(val)
    return tot_val

In [ ]:
tot_val = detect(news)

print(tot_val)

ABB Q2 PAT seen up 48.1% YoY to Rs. 177.7 cr: Prabhudas Lilladher ---- positive
Tube Investment, ABB, Varun Bev, IHCL likely to enter MSCI index ---- neutral
#######################################################
Artson Engineering spurts after bagging order worth Rs 42.76 cr ---- positive
Street signs: Mkts to feel selling pressure, Adani Enterprises stock & more ---- negative
Movers & Shakers: Stocks that will see action this week ---- neutral
#######################################################
Adani Green Energy shares rise after firm buys three SPVs from Inox Green Energy ---- positive
Adani Green Energy completes acquisition of three SPVs from Inox Green ---- neutral
Inox Wind transfers 3 SPVs to Adani Green Energy; stock zooms ~2.5% ---- neutral
#######################################################
Taking Stock | Market ends lower ahead of Fed meet outcome; FMCG gains, power stocks drag ---- positive
Stocks in news: TCS, Adani Ports, Bajaj Auto, Infosys and more ---- neutr

In [ ]:
def get_sent(val):
    agg = 0
    for i in val:
        if i == 'positive':
            agg = agg + 1
        elif i == 'negative':
            agg = agg - 1

    if agg > 0:
        return('positive')
    elif agg < 0:
        return('negative')
    else:
        return('neutral')

In [ ]:
sent = list()
for i in tot_val:
    sent.append(get_sent(i))

In [ ]:
sent

['positive',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'positive',
 'negative',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'negative',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'positive',
 'negative',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'positive',
 'neutral',
 'positive',
 'negative',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'positive',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'positive',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'positive',
 'positive',


In [ ]:
tick_df['sentiment'] = sent 

tick_df.head(20)

,stock,tick,sentiment
0,ABB India Ltd,ABB,positive
1,Adani Enterprises Ltd,ADEL,neutral
2,Adani Green Energy Ltd,ADNA,positive
3,Adani Ports and Special Economic Zone Ltd,APSE,positive
4,Adani Power Ltd,ADAN,neutral
5,Adani Total Gas Ltd,ADAG,neutral
6,Adani Transmission Ltd,ADAI,neutral
7,Adani Wilmar Ltd,AWL,neutral
8,Ambuja Cements Ltd,ABUJ,neutral
9,Apollo Hospitals Enterprise Ltd,APLH,negative


In [ ]:
tick_df

,stock,tick,sentiment
0,ABB India Ltd,ABB,positive
1,Adani Enterprises Ltd,ADEL,neutral
2,Adani Green Energy Ltd,ADNA,positive
3,Adani Ports and Special Economic Zone Ltd,APSE,positive
4,Adani Power Ltd,ADAN,neutral
...,...,...,...
95,United Spirits Ltd,UNSP,neutral
96,Varun Beverages Ltd,VARB,neutral
97,Vedanta Ltd,VDAN,negative
98,Wipro Ltd,WIPR,neutral
